# Assess generalizability of the model by using drug dose curve data

**NOTE:** For assess generalizability based on a drug dose response, we will be using Plates 1 and 2, split by heart number (all failing hearts).

## Import libraries

In [1]:
import pathlib
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import seaborn as sns
from joblib import load
from sklearn.metrics import precision_recall_curve

sys.path.append("../utils")
from eval_utils import generate_confusion_matrix_df, generate_f1_score_df
from training_utils import get_X_y_data

## Set paths and variables

In [2]:
# Directory with plate datasets
data_dir = pathlib.Path("../3.process_cfret_features/data/single_cell_profiles")

# Directory with models
models_dir = pathlib.Path("./models")

# Directory for model figures output
fig_dir = pathlib.Path("./figures")
fig_dir.mkdir(exist_ok=True)

# Directory for probability data to be saved
prob_dir = pathlib.Path("./prob_data")
prob_dir.mkdir(exist_ok=True)

# Load in each model individually
final_model = load(
    pathlib.Path(f"{models_dir}/log_reg_fs_plate_4_final_downsample.joblib")
)
shuffled_model = load(
    pathlib.Path(f"{models_dir}/log_reg_fs_plate_4_shuffled_downsample.joblib")
)

## Load in Plate 4 fs data to extract column names to filter from the other plates

In [3]:
# Load in Plate 4 normalized feature selected data metadata (used with model) to get the feature columns to filter the plate data
parquet_metadata = pq.read_metadata(
    pathlib.Path(f"{data_dir}/localhost231120090001_sc_feature_selected.parquet")
)

# Get the column names from the metadata
all_column_names = parquet_metadata.schema.names

# Filter out the column names that start with "Metadata_"
model_column_names = [
    col for col in all_column_names if not col.startswith("Metadata_")
]

print(len(model_column_names))
print(model_column_names)

638
['Cytoplasm_AreaShape_Compactness', 'Cytoplasm_AreaShape_Eccentricity', 'Cytoplasm_AreaShape_Extent', 'Cytoplasm_AreaShape_FormFactor', 'Cytoplasm_AreaShape_MajorAxisLength', 'Cytoplasm_AreaShape_MinorAxisLength', 'Cytoplasm_AreaShape_Perimeter', 'Cytoplasm_AreaShape_Solidity', 'Cytoplasm_AreaShape_Zernike_0_0', 'Cytoplasm_AreaShape_Zernike_1_1', 'Cytoplasm_AreaShape_Zernike_2_0', 'Cytoplasm_AreaShape_Zernike_2_2', 'Cytoplasm_AreaShape_Zernike_3_1', 'Cytoplasm_AreaShape_Zernike_3_3', 'Cytoplasm_AreaShape_Zernike_4_0', 'Cytoplasm_AreaShape_Zernike_4_2', 'Cytoplasm_AreaShape_Zernike_5_1', 'Cytoplasm_AreaShape_Zernike_5_3', 'Cytoplasm_AreaShape_Zernike_6_0', 'Cytoplasm_AreaShape_Zernike_6_2', 'Cytoplasm_AreaShape_Zernike_6_4', 'Cytoplasm_AreaShape_Zernike_7_1', 'Cytoplasm_AreaShape_Zernike_7_3', 'Cytoplasm_AreaShape_Zernike_8_0', 'Cytoplasm_AreaShape_Zernike_8_2', 'Cytoplasm_AreaShape_Zernike_8_4', 'Cytoplasm_AreaShape_Zernike_8_6', 'Cytoplasm_AreaShape_Zernike_9_1', 'Cytoplasm_AreaSh

## Load in Plates 1 and 2, concat vertically, and drop any rows where there are NaNs in the feature columns from the model

In [4]:
# Load in Plate 1 and 2 data -> concat
plate_1_df = pd.read_parquet(
    pathlib.Path(f"{data_dir}/localhost220512140003_KK22-05-198_sc_normalized.parquet")
)
plate_2_df = pd.read_parquet(
    pathlib.Path(
        f"{data_dir}/localhost220513100001_KK22-05-198_FactinAdjusted_sc_normalized.parquet"
    )
)

# Concat separate parts of the same plate together
concatenated_df = pd.concat([plate_1_df, plate_2_df], axis=0)

# Drop rows with NaN values in feature columns that the model uses
concatenated_df = concatenated_df.dropna(subset=model_column_names)

print(concatenated_df.shape)
concatenated_df.head()

(61174, 2022)


,Metadata_WellRow,Metadata_WellCol,Metadata_heart_number,Metadata_treatment,Metadata_dose,Metadata_dose_unit,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_Cells_Location_Center_X,Metadata_Cells_Location_Center_Y,...,Nuclei_Texture_Variance_Hoechst_3_02_256,Nuclei_Texture_Variance_Hoechst_3_03_256,Nuclei_Texture_Variance_Mitochondria_3_00_256,Nuclei_Texture_Variance_Mitochondria_3_01_256,Nuclei_Texture_Variance_Mitochondria_3_02_256,Nuclei_Texture_Variance_Mitochondria_3_03_256,Nuclei_Texture_Variance_PM_3_00_256,Nuclei_Texture_Variance_PM_3_01_256,Nuclei_Texture_Variance_PM_3_02_256,Nuclei_Texture_Variance_PM_3_03_256
0,A,1,3,drug_x,5.0,uM,506.565460,27.116063,509.949487,25.078927,...,-0.058183,-0.053958,-0.378350,-0.378093,-0.381519,-0.374658,-0.202587,-0.210964,-0.210605,-0.199854
1,A,1,3,drug_x,5.0,uM,220.491103,103.878114,238.995447,104.179514,...,-0.165152,-0.165087,-0.374914,-0.372956,-0.378561,-0.373619,-0.247781,-0.247595,-0.248622,-0.247597
2,A,1,3,drug_x,5.0,uM,180.302253,171.314685,199.191002,189.827676,...,-0.249895,-0.257507,-0.382185,-0.380423,-0.384313,-0.378697,-0.220673,-0.223056,-0.221877,-0.217614
3,A,1,3,drug_x,5.0,uM,377.832677,247.411417,363.699578,247.405714,...,-0.027866,-0.034019,-0.143649,-0.109237,-0.117521,-0.152168,0.213568,0.239172,0.218580,0.196728
4,A,1,3,drug_x,5.0,uM,533.028105,256.360834,550.957839,243.532560,...,-0.198482,-0.198946,-0.380414,-0.377410,-0.381325,-0.375556,-0.247209,-0.248295,-0.247855,-0.246887


## Filter the concat data to only include metadata and filtered feature columns

In [5]:
# Extract metadata columns from the plate
metadata_columns = [col for col in concatenated_df.columns if col.startswith("Metadata_")]

# Extract feature columns that don't start with "Metadata_"
feature_columns = [col for col in concatenated_df.columns if not col.startswith("Metadata_")]

# Filter columns in data frame to only include those in the model
filtered_feature_columns = [
    col for col in concatenated_df.columns if col in model_column_names
]

# Filter the DataFrame to keep only the desired columns
concatenated_df = concatenated_df[metadata_columns + filtered_feature_columns]

concatenated_df

,Metadata_WellRow,Metadata_WellCol,Metadata_heart_number,Metadata_treatment,Metadata_dose,Metadata_dose_unit,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_Cells_Location_Center_X,Metadata_Cells_Location_Center_Y,...,Nuclei_Texture_InverseDifferenceMoment_ER_3_01_256,Nuclei_Texture_InverseDifferenceMoment_ER_3_03_256,Nuclei_Texture_InverseDifferenceMoment_Mitochondria_3_00_256,Nuclei_Texture_InverseDifferenceMoment_Mitochondria_3_02_256,Nuclei_Texture_InverseDifferenceMoment_PM_3_01_256,Nuclei_Texture_InverseDifferenceMoment_PM_3_03_256,Nuclei_Texture_SumVariance_ER_3_03_256,Nuclei_Texture_SumVariance_Hoechst_3_03_256,Nuclei_Texture_SumVariance_Mitochondria_3_03_256,Nuclei_Texture_SumVariance_PM_3_01_256
0,A,1,3,drug_x,5.0,uM,506.565460,27.116063,509.949487,25.078927,...,0.216560,0.720942,1.707297,1.788489,0.375038,0.915553,-0.241447,-0.082324,-0.350832,-0.206218
1,A,1,3,drug_x,5.0,uM,220.491103,103.878114,238.995447,104.179514,...,0.771378,0.179252,1.850895,1.402105,0.827124,0.596812,-0.325701,-0.201034,-0.350992,-0.236428
2,A,1,3,drug_x,5.0,uM,180.302253,171.314685,199.191002,189.827676,...,0.434554,0.787391,1.565268,1.423320,0.856107,0.994298,-0.292311,-0.257731,-0.357104,-0.210559
3,A,1,3,drug_x,5.0,uM,377.832677,247.411417,363.699578,247.405714,...,0.541023,0.109774,0.821423,0.660647,-0.436415,0.032893,-0.165404,-0.112353,-0.144398,0.197364
4,A,1,3,drug_x,5.0,uM,533.028105,256.360834,550.957839,243.532560,...,0.873848,0.388526,1.706142,0.912066,0.842757,0.861225,-0.316048,-0.213081,-0.355399,-0.237526
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17990,H,12,9,DMSO,0.0,None,974.718206,798.326649,948.166085,760.015505,...,0.203443,0.373291,1.199331,1.137848,0.657579,0.655504,-0.170523,-0.318980,-0.179057,-0.198828
17991,H,12,9,DMSO,0.0,None,330.699948,931.530264,338.359040,969.872148,...,-1.346656,-1.827966,0.373664,0.165008,-0.597820,-1.075883,1.889800,-0.256832,-0.168293,0.363226
17992,H,12,9,DMSO,0.0,None,505.437850,933.301232,492.944089,929.988103,...,0.532556,-0.781118,0.183586,0.276342,-0.066746,-0.180104,0.587139,-0.194416,-0.129824,-0.171903
17993,H,12,9,DMSO,0.0,None,973.120087,895.116266,973.120087,895.116266,...,-0.523495,0.791860,1.230681,1.009320,0.501782,0.555003,0.050046,-0.299357,-0.184335,-0.198898


## Create a dictionary with concat dataframe splitting the data by the heart number

In [6]:
# Split the plate data into different data frames with different data for applying the model to

# Define a dictionary
plate_3_dfs_dict = {}

# Filter the DataFrame to a data frame per treatment
three_df = concatenated_df[concatenated_df["Metadata_heart_number"] == 3]
eight_df = concatenated_df[concatenated_df["Metadata_heart_number"] == 8]
nine_df = concatenated_df[concatenated_df["Metadata_heart_number"] == 9]

# Add each DataFrame to the dictionary with a corresponding key
plate_3_dfs_dict["heart_3"] = {"data_df": three_df}
plate_3_dfs_dict["heart_8"] = {"data_df": eight_df}
plate_3_dfs_dict["heart_9"] = {"data_df": nine_df}

## Extract final model predicted probabilities for each heart number

In [7]:
# Create an empty DataFrame to store the results
combined_prob_df = pd.DataFrame()

for model_path in models_dir.iterdir():
    model_type = model_path.stem.split("_")[5]  # Get the model type

    for data, info in plate_3_dfs_dict.items():
        # Ensure that the file is named the correct data split
        data_split = data
        print(f"Extracting {model_type} probabilities from {data} data...")

        # Load in model to apply to datasets
        model = load(model_path)

        # Load in label encoder
        le = load(
            pathlib.Path("./encoder_results/label_encoder_log_reg_fs_plate_4.joblib")
        )

        # Get unique cell types and their corresponding encoded values
        unique_labels = le.classes_
        encoded_values = le.transform(unique_labels)

        # Create a dictionary mapping encoded values to original labels
        label_dict = dict(zip(encoded_values, unique_labels))

        # Load in data frame associated with the data split
        data_df = info["data_df"].reset_index(drop=True)

        # Load in X data to get predicted probabilities
        X, _ = get_X_y_data(df=data_df, label="Metadata_heart_number")

        # Predict class probabilities for morphology feature data
        predicted_probs = model.predict_proba(X)

        # Storing probabilities in a pandas DataFrame
        prob_df = pd.DataFrame(predicted_probs, columns=model.classes_)

        # Update column names in prob_df using the dictionary and add suffix "_probas"
        prob_df.columns = [label_dict[col] + "_probas" for col in prob_df.columns]

        # Add a new column called predicted_label for each row
        prob_df["predicted_label"] = prob_df.apply(
            lambda row: row.idxmax()[:-7], axis=1
        )

        # Select metadata columns from the data
        metadata_columns = data_df.filter(like="Metadata_")

        # Combine metadata columns with predicted probabilities DataFrame based on index
        prob_df = prob_df.join(metadata_columns)

        # Add a new column for model_type
        prob_df["model_type"] = model_type

        # Append the probability DataFrame to the combined DataFrame
        combined_prob_df = pd.concat([combined_prob_df, prob_df], ignore_index=True)

# Save combined prob data
combined_prob_df.to_csv(f"{prob_dir}/combined_plates_1_2_predicted_proba.csv", index=False)

Extracting shuffled probabilities from heart_3 data...
Extracting shuffled probabilities from heart_8 data...
Extracting shuffled probabilities from heart_9 data...
Extracting final probabilities from heart_3 data...
Extracting final probabilities from heart_8 data...
Extracting final probabilities from heart_9 data...
